# ASSOCIATION RULE MINING USING APRIORI ALGORITHM

This notebook contains step-by-step implementation of the apriori algorithm using python's `mlxtend` to generate association rules for online retail transactions

### About dataset

The online retail dataset was sourced from kaggle.com. It contains the following columns:
1. InvoiceNo - Invoice number. A 6-digit integral umber uniquely assigned to each transaction. Some transaction contains "C"(meaning a cancekked transaction) and "B"(an adjusted bad debt)
2. StockCode - Product code. A 5-digit integral number uniquely assigned to each distinct product
3. Description - the product name
4. Quantity - the quantities of each product per transaction
5. InvoiceDate - the invoice data and time. The day and time when each transaction was generated
6. UnitPrice - the product price per unit in sterling
7. CustomerID - the customer number. The 5-digit integral number uniquely assigned to each customer
8. Country - the country name, the country where each customer resides

`Goal` - analyzing the items customers purchase together to understand their purchasing habits and preferences.

In [ ]:
# connect to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import necessary libraries
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

#### Data Loading and Data Understanding

In [ ]:
file_path = '/content/drive/MyDrive/SCO415/Online_Retail.csv'

In [ ]:
df = pd.read_csv(file_path, encoding='latin-1')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom


In [ ]:
df.shape

(541909, 8)

The dataset contains 8 features and 541909 instances

In [ ]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


From the `describe()` function, the numerical features show that:

- the customer id contains missing values
- the minimum values of the product quantity and unit price are negative values - meaning the dataset contains reversed transactions

In [ ]:
# clean dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


From the `info()` function:

- the `Quantity` and `CustomerID` contains missing values
- some features contain object, integral and floating datatypes


#### Data Preprocessing

##### Data Cleaning

In [ ]:
df.isnull().sum()

,0
InvoiceNo,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,135080
Country,0


the dataset contains some missing values. The following will be done:
- drop the missing descriptions because they are more important
- drop the whole customer id column, since this is just a demo data and won't be integrated into any existing application

In [ ]:
# drop missing descriptions
df = df.dropna(subset=['Description'])
df.isnull().sum()

,0
InvoiceNo,0
StockCode,0
Description,0
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,133626
Country,0


In [ ]:
df = df.drop(columns='CustomerID')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,United Kingdom


In [ ]:
invoice_no = df['InvoiceNo'].value_counts()
invoice_no

,count
InvoiceNo,
573585,1114
581219,749
581492,731
580729,721
558475,705
...,...
C562185,1
562164,1
C562160,1


In the **InvoiceNo** column there exits instances that start with `C`. These invoices will be removed because they indicate `Credit Notes`.

Credit notes represents refunds or returns, they reflect reversal of a transaction. If they are left in the dataset when generating rules they might mislead the output

In [ ]:
# sample on how we can remove the credit invoices
demo_df = pd.DataFrame({'A': ['C123', '5324', 'C456', '7864']})

mask = demo_df['A'].str.startswith('C')
print(mask)

0     True
1    False
2     True
3    False
Name: A, dtype: bool


In [ ]:
print(~mask)

0    False
1     True
2    False
3     True
Name: A, dtype: bool


In [ ]:
print(demo_df[~mask])

      A
1  5324
3  7864


~ flips boolean, used to exclude rows that meet a condition

In [ ]:
# remove credit notes
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,United Kingdom
...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/11 12:50,0.85,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/11 12:50,2.10,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/11 12:50,4.15,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/11 12:50,4.15,France


In the **InvoiceNo** column there exists instances starting with `A`. These invoices will also be removed because they indicate `Adjusted Bad Debt`

Adjusted Bad Debt means internal accounting adjustments, not actual customer purchases because they are used to reverse unpaid or problematic transations or handle situations where payment failed or had to be manually corrected

We will have to filter out the descriptions with "adjusted bad debt" instance

In [ ]:
df = df[~df['InvoiceNo'].astype(str).str.startswith('A')]
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,United Kingdom
...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/11 12:50,0.85,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/11 12:50,2.10,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/11 12:50,4.15,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/11 12:50,4.15,France


In [ ]:
# check duplicates and remove them
df.duplicated().sum()

np.int64(5231)

In [ ]:
df.drop_duplicates(inplace=True)
df.duplicated().sum()

<ipython-input-18-da2a6d66ab19>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


np.int64(0)

In [ ]:
countries = df['Country'].value_counts()
countries

,count
Country,
United Kingdom,481022
Germany,9027
France,8393
EIRE,7883
Spain,2480
Netherlands,2363
Belgium,2031
Switzerland,1959
Portugal,1492


**United Kingdom** has the highest number of transactions, followed by **Germany**

We can start by creating basket for the majority

In [ ]:
df = df[df['Country'] == 'United Kingdom']
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,United Kingdom
...,...,...,...,...,...,...,...
541889,581585,22466,FAIRY TALE COTTAGE NIGHT LIGHT,12,12/9/11 12:31,1.95,United Kingdom
541890,581586,22061,LARGE CAKE STAND HANGING STRAWBERY,8,12/9/11 12:49,2.95,United Kingdom
541891,581586,23275,SET OF 3 HANGING OWLS OLLIE BEAK,24,12/9/11 12:49,1.25,United Kingdom
541892,581586,21217,RED RETROSPOT ROUND CAKE TINS,24,12/9/11 12:49,8.95,United Kingdom


In [ ]:
# exclude irrelevant keywords
exclude_keywords = [
    'wrongly', 'adjust', 'manual', 'error', 'sold in set', 'stock check',
    'test', 'throw away', 'thrown away', 'can\'t sell', 'push order',
    'water damage', 'website fixed', 'wet', 'wrong', 'sold', 'smashed', 'samples'
]

pattern = '|'.join(exclude_keywords)
df = df[~df['Description'].str.contains(pattern)]

##### Feature Engineering

**Create Baskets**

Using a pivot operation to encode each transaction as a row, with iteams as columns

In [ ]:
# group by invoice and description, and quantity
basket = df.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().fillna(0)
basket

Description,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TOADSTOOL BEDSIDE LIGHT,...,mix up with c,mixed up,mouldy,"mouldy, unsaleable.",mystery! Only ever imported 1800,on cargo order,rcvd be air temp fix for dotcom sit,re dotcom quick fix.,returned,showroom
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581584,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# converting quantities to 1s and 0s
basket_encoded = basket.applymap(lambda x: 1 if x > 0 else 0)
basket_encoded

<ipython-input-23-40fe816d55d6>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket_encoded = basket.applymap(lambda x: 1 if x > 0 else 0)


Description,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TOADSTOOL BEDSIDE LIGHT,...,mix up with c,mixed up,mouldy,"mouldy, unsaleable.",mystery! Only ever imported 1800,on cargo order,rcvd be air temp fix for dotcom sit,re dotcom quick fix.,returned,showroom
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581582,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581583,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581584,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# generate frequent itemsets
frequent_itemsets = apriori(basket_encoded, min_support=0.03, use_colnames=True)
frequent_itemsets

/usr/local/lib/python3.11/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.046131,(6 RIBBONS RUSTIC CHARM)
1,0.031348,(60 CAKE CASES VINTAGE CHRISTMAS)
2,0.040628,(60 TEATIME FAIRY CAKE CASES)
3,0.047264,(ALARM CLOCK BAKELIKE GREEN)
4,0.035394,(ALARM CLOCK BAKELIKE PINK)
...,...,...
126,0.030754,"(JUMBO BAG RED RETROSPOT, JUMBO BAG BAROQUE B..."
127,0.042355,"(JUMBO BAG PINK POLKADOT, JUMBO BAG RED RETROS..."
128,0.035448,"(JUMBO SHOPPER VINTAGE RED PAISLEY, JUMBO BAG ..."
129,0.037661,"(JUMBO STORAGE BAG SUKI, JUMBO BAG RED RETROSPOT)"


In [ ]:
# generate association rules
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(ALARM CLOCK BAKELIKE RED ),(ALARM CLOCK BAKELIKE GREEN),0.050178,0.047264,0.030377,0.605376,12.808271,1.0,0.028005,2.414289,0.970630,0.452936,0.585799,0.624035
1,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED ),0.047264,0.050178,0.030377,0.642694,12.808271,1.0,0.028005,2.658288,0.967661,0.452936,0.623818,0.624035
2,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.037930,0.050394,0.031132,0.820768,16.287063,1.0,0.029221,5.298199,0.975607,0.544340,0.811257,0.719271
3,(GREEN REGENCY TEACUP AND SAUCER),(PINK REGENCY TEACUP AND SAUCER),0.050394,0.037930,0.031132,0.617773,16.287063,1.0,0.029221,2.517012,0.988411,0.544340,0.602703,0.719271
4,(ROSES REGENCY TEACUP AND SAUCER ),(GREEN REGENCY TEACUP AND SAUCER),0.051635,0.050394,0.037822,0.732497,14.535446,1.0,0.035220,3.549895,0.981903,0.589076,0.718302,0.741516
5,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER ),0.050394,0.051635,0.037822,0.750535,14.535446,1.0,0.035220,3.801601,0.980620,0.589076,0.736953,0.741516
6,(JUMBO BAG RED RETROSPOT),(JUMBO BAG BAROQUE BLACK WHITE),0.104565,0.049099,0.030754,0.294118,5.990304,1.0,0.025620,1.347110,0.930345,0.250219,0.257670,0.460246
7,(JUMBO BAG BAROQUE BLACK WHITE),(JUMBO BAG RED RETROSPOT),0.049099,0.104565,0.030754,0.626374,5.990304,1.0,0.025620,2.396607,0.876078,0.250219,0.582743,0.460246
8,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.062534,0.104565,0.042355,0.677308,6.477413,1.0,0.035816,2.774892,0.902024,0.339533,0.639626,0.541182
9,(JUMBO BAG RED RETROSPOT),(JUMBO BAG PINK POLKADOT),0.104565,0.062534,0.042355,0.405057,6.477413,1.0,0.035816,1.575724,0.944364,0.339533,0.365371,0.541182


In [ ]:
rules[(rules['lift'] >= 6) &
      (rules['confidence'] >= 0.6)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(ALARM CLOCK BAKELIKE RED ),(ALARM CLOCK BAKELIKE GREEN),0.050178,0.047264,0.030377,0.605376,12.808271,1.0,0.028005,2.414289,0.970630,0.452936,0.585799,0.624035
1,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED ),0.047264,0.050178,0.030377,0.642694,12.808271,1.0,0.028005,2.658288,0.967661,0.452936,0.623818,0.624035
2,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.037930,0.050394,0.031132,0.820768,16.287063,1.0,0.029221,5.298199,0.975607,0.544340,0.811257,0.719271
3,(GREEN REGENCY TEACUP AND SAUCER),(PINK REGENCY TEACUP AND SAUCER),0.050394,0.037930,0.031132,0.617773,16.287063,1.0,0.029221,2.517012,0.988411,0.544340,0.602703,0.719271
4,(ROSES REGENCY TEACUP AND SAUCER ),(GREEN REGENCY TEACUP AND SAUCER),0.051635,0.050394,0.037822,0.732497,14.535446,1.0,0.035220,3.549895,0.981903,0.589076,0.718302,0.741516
5,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER ),0.050394,0.051635,0.037822,0.750535,14.535446,1.0,0.035220,3.801601,0.980620,0.589076,0.736953,0.741516
8,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.062534,0.104565,0.042355,0.677308,6.477413,1.0,0.035816,2.774892,0.902024,0.339533,0.639626,0.541182
